# Run This Notebook

<h3><a href="https://hub.openveda.cloud/hub/user-redirect/git-pull?repo=https://github.com/NASA-IMPACT/veda-docs&urlpath=lab/tree/veda-docs/user-guide/notebooks/tutorials/fancy-mapping-tutorial.ipynb&branch=main">🚀 Launch in VEDA JupyterHub (requires access)</a></h3>

<h4> To obtain credentials to VEDA Hub, <a href = "https://docs.openveda.cloud/user-guide/scientific-computing/getting-access.html"> follow this link for more information.</a></h4>

<div class="alert alert-block" style="
     background-color: #f8d7da;
     color: #721c24;
     border-left: 4px solid #28a745;
  ">
Disclaimer: it is highly recommended to run a tutorial within NASA VEDA JupyterHub, which already includes functions for processing and visualizing data specific to VEDA stories. Running the tutorial outside of the VEDA JupyterHub may lead to errors, specifically related to EarthData authentication. Additionally, it is recommended to use the Pangeo workspace within the VEDA JupyterHub, since certain packages relevant to this tutorial are already installed. </div>

<h4> If you <strong>do not</strong> have a VEDA Jupyterhub Account you can launch this notebook on your local environment using MyBinder by clicking the icon below.</h4>
<br/>
<a href="https://binder.openveda.cloud/v2/gh/NASA-IMPACT/veda-docs/9c8cdbae92906fb7062b8a0c759dad90e223a4f9?urlpath=lab%2Ftree%2Fuser-guide%2Fnotebooks%2Ftutorials%2Ffancy-mapping-tutorial.ipynb">
<img src="https://binder.openveda.cloud/badge_logo.svg" alt="Binder" title="A cute binder" width="150"/> </a>

## Environment Setup

If running this notebook outside of the VEDA JupyterHub, install the following packages:

In [3]:
# Load libraries

import requests
import matplotlib.pyplot as plt
import plotutils as putils
from pystac_client import Client
import folium

In [4]:
# For retrieving data already catalogued in VEDA STAC
STAC_API_URL = "https://openveda.cloud/api/stac"
RASTER_API_URL = "https://openveda.cloud/api/raster"

# Open STAC client designed for interacting with SpatioTemporal Asset Catalog (STAC) APIs and Catalogs
client_STAC = Client.open(STAC_API_URL)

This example will show how to create an interactive mapping experience similar to what is used in the VEDA environment. Using an example dataset from the VEDA STAC catalog, this notebook utilizes the Folium package to create this experience. 

## Processing steps:
1.) Choose collection ID from the STAC catalog and date for analysis<br />
2.) Retrieve collection information and items from VEDA STAC catalog<br />
3.) Retrieve item statistics and tiling information<br />
4.) Plot data<br />

## Choose variable and retrieve json from VEDA STAC catalogue

In [7]:
collection_id = "landslides-ndvi"
date = "2024-10-12"

results = client_STAC.search(collections=[collection_id], datetime=date)

# ── VEDA Collection Request ─────────────────────────────────────────────────────────────────────────────────────

items = list(results.items())
assert len(items) != 0, "No items found"
item = items[0]
collection = item.get_collection()

# grab the dashboard render block
dashboard_render = collection.extra_fields["renders"]["dashboard"]

assets = dashboard_render["assets"][0]
((vmin, vmax),) = dashboard_render["rescale"]

collection

<CollectionClient id=landslides-ndvi>

In [8]:
# ── VEDA Tile Request ─────────────────────────────────────────────────────────────────────────────────────
colormap_name = "coolwarm_r"

# Build endpoint URL without worrying about trailing slashes
response = requests.get(
    f"{RASTER_API_URL.rstrip('/')}/collections/{collection_id}"
    f"/items/{item.id}/WebMercatorQuad/tilejson.json?"
    f"&assets={assets}"
    f"&color_formula=gamma+r+1.05&colormap_name={colormap_name}"
    f"&rescale={vmin},{vmax}",
)

response.raise_for_status()

tiles = response.json()
print(tiles)

{'tilejson': '2.2.0', 'version': '1.0.0', 'scheme': 'xyz', 'tiles': ['https://openveda.cloud/api/raster/collections/landslides-ndvi/items/S2_NDVI_Diff_2024-10-12/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=cog_default&color_formula=gamma+r+1.05&colormap_name=coolwarm_r&rescale=-0.5%2C0.5'], 'minzoom': 0, 'maxzoom': 24, 'bounds': [-84.37131820919367, 34.20966703314409, -80.89024067922162, 37.04653228059104], 'center': [-82.63077944420765, 35.62809965686756, 0]}


## Plot data

In [ ]:
# First we will present the different basemaps that we have access to underlay beneath our tile requests
# For the first map, we will utilize the 'esri-satellite-labels' map layer
putils.get_available_basemaps()

In [10]:
# Use the new plot_folium_from_VEDA_STAC function
m = putils.plot_folium_from_VEDA_STAC(
    tiles_url_template=tiles["tiles"][0],
    center_coords=[35.44368, -82.26868],
    zoom_level=12.5,
    rescale=(vmin, vmax),
    colormap_name=colormap_name,
    capitalize_cmap=False,  # to better match VEDA colors and matplotlib colors
    layer_name="Sentinel 2 NDVI Difference: Hurricane Helene",
    date=f"{date}T00:00:00Z",
    colorbar_caption="Normalized Difference Vegetation Index (Difference)",
    attribution="Sentinel 2 NDVI Difference: Hurricane Helene",
    tile_name="Sentinel 2 NDVI Difference: Hurricane Helene",
    opacity=0.8,
    height="800px",
    basemap_style="cartodb-positron"  # Use light basemap for good contrast
)

print(
    "Visualization of vegetation damage done by Hurricane Helene in western North Caroline in September 2024."
)
# Display the map
m

Visualization of vegetation damage done by Hurricane Helene in western North Caroline in September 2024.
